In [348]:
from cassandra.cluster import Cluster
import uuid
from datetime import datetime, timedelta

In [125]:
cluster = Cluster(['localhost'], protocol_version=4)

In [126]:
session = cluster.connect()

In [127]:
query = """
    CREATE KEYSPACE IF NOT EXISTS test_keyspace 
    WITH replication = {
        'class': 'SimpleStrategy', 
        'replication_factor': '1'
    }
"""
session.execute(query)

In [128]:
session.set_keyspace('test_keyspace')

In [129]:
items = [
    {
        "category": "Phone",
        "model": "iPhone 13",
        "producer": "Apple",
        "price": 1200
    },
    {
        "category": "Phone",
        "model": "iPhone 12",
        "producer": "Apple",
        "price": 1000
    },
    {
        "category": "Phone",
        "model": "Galaxy S26",
        "producer": "Samsung",
        "price": 1150
    },
    {
        "category": "TV",
        "model": "SmartTV v4.2",
        "producer": "Samsung",
        "price": 1200,
        "custom_params":{"sub_category": "Smart TV"},
    },
    {
        "category": "TV",
        "model": "Gigantic 1000",
        "producer": "Spacio",
        "price": 2000,
        "custom_params": {"max_discount": "25%"}
    },
    {
        "category": "Cooker",
        "model": "Pasteroni",
        "producer": "Italiez",
        "price": 2000,
        "custom_params": {"height": "38cm"}
    },
    {
        "category": "Cooker",
        "model": "Pasteroni Pro",
        "producer": "Italiez",
        "price": 2500,
        "custom_params": {"height": "40cm"}
    },
]

In [301]:
query = """
    DROP MATERIALIZED VIEW IF EXISTS items_by_price 
"""
session.execute(query)
query = """
    DROP MATERIALIZED VIEW IF EXISTS items_by_price_and_producer
"""
session.execute(query)

query = """
    DROP TABLE IF EXISTS items
"""
result = session.execute(query)

In [302]:
query = """
    CREATE TABLE IF NOT EXISTS items (
        id UUID,
        category text,
        model text,
        producer text,
        price int,
        custom_params map<text,text>,
        PRIMARY KEY (category, price)
    ) WITH CLUSTERING ORDER BY (price ASC);
"""
session.execute(query)

In [303]:
query = """
    CREATE INDEX IF NOT EXISTS ON items (ENTRIES(custom_params))
"""
session.execute(query)
query = """
    CREATE INDEX IF NOT EXISTS ON items (KEYS(custom_params))
"""
session.execute(query)

In [304]:
query_with_custom_params = """
    INSERT INTO items (id, category, model, producer, price, custom_params)
    VALUES (%s, %s, %s, %s, %s, %s)
"""
query_without_custom_params = """
    INSERT INTO items (id, category, model, producer, price)
    VALUES (%s, %s, %s, %s, %s)
"""

In [305]:
for item in items:
    if 'custom_params' in item:
        session.execute(query_with_custom_params, (uuid.uuid1(), *item.values()))
    else:
        session.execute(query_without_custom_params, (uuid.uuid1(), *item.values()))

In [306]:
query = """
    DESCRIBE table items
"""
result = session.execute(query)

In [307]:
for row in result.all():
    print(row)

Row(keyspace_name='test_keyspace', type='table', name='items', create_statement="CREATE TABLE test_keyspace.items (\n    category text,\n    price int,\n    id uuid,\n    model text,\n    producer text,\n    custom_params map<text, text>,\n    PRIMARY KEY (category, price)\n) WITH CLUSTERING ORDER BY (price ASC)\n    AND additional_write_policy = '99p'\n    AND bloom_filter_fp_chance = 0.01\n    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}\n    AND cdc = false\n    AND comment = ''\n    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}\n    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}\n    AND memtable = 'default'\n    AND crc_check_chance = 1.0\n    AND default_time_to_live = 0\n    AND extensions = {}\n    AND gc_grace_seconds = 864000\n    AND max_index_interval = 2048\n    AND memtable_flush_period_in_ms = 0\n    AND mi

In [308]:
query = """
    select * 
    from items
    where category = 'Phone'
    order by price
"""
result = session.execute(query)

In [309]:
for row in result.all():
    print(row)

Row(category='Phone', price=1000, custom_params=None, id=UUID('f1b2c097-dddb-11ee-845d-1062e54e838c'), model='iPhone 12', producer='Apple')
Row(category='Phone', price=1150, custom_params=None, id=UUID('f1b3861b-dddb-11ee-8604-1062e54e838c'), model='Galaxy S26', producer='Samsung')
Row(category='Phone', price=1200, custom_params=None, id=UUID('f1b17adc-dddb-11ee-b6a4-1062e54e838c'), model='iPhone 13', producer='Apple')


In [310]:
query = """
    CREATE INDEX IF NOT EXISTS ON items (model)
"""
session.execute(query)

In [313]:
query = """
    select * 
    from items
    where model = 'Galaxy S26' 
"""
result = session.execute(query)

In [314]:
for row in result.all():
    print(row)

Row(category='Phone', price=1150, custom_params=None, id=UUID('f1b3861b-dddb-11ee-8604-1062e54e838c'), model='Galaxy S26', producer='Samsung')


In [315]:
query = """
    DROP MATERIALIZED VIEW IF EXISTS items_by_price 
"""
session.execute(query)

query = """
    CREATE MATERIALIZED VIEW items_by_price 
    AS SELECT * 
    FROM items 
    WHERE price >= 1000 and price <= 1500 and category is not null and id is not null
    PRIMARY KEY (category, price, id)
"""
result = session.execute(query)

In [316]:
query = """
    select * 
    from items_by_price
"""
result = session.execute(query)

In [317]:
for row in result.all():
    print(row)

Row(category='Phone', price=1000, id=UUID('f1b2c097-dddb-11ee-845d-1062e54e838c'), custom_params=None, model='iPhone 12', producer='Apple')
Row(category='Phone', price=1150, id=UUID('f1b3861b-dddb-11ee-8604-1062e54e838c'), custom_params=None, model='Galaxy S26', producer='Samsung')
Row(category='Phone', price=1200, id=UUID('f1b17adc-dddb-11ee-b6a4-1062e54e838c'), custom_params=None, model='iPhone 13', producer='Apple')
Row(category='TV', price=1200, id=UUID('f1b449f2-dddb-11ee-be16-1062e54e838c'), custom_params=OrderedMapSerializedKey([('sub_category', 'Smart TV')]), model='SmartTV v4.2', producer='Samsung')


In [318]:
query = """
    CREATE INDEX IF NOT EXISTS ON items (price)
"""
session.execute(query)

In [319]:
query = """
    select * 
    from items
    where price = 1000
"""
result = session.execute(query)

In [320]:
query = """
    DROP MATERIALIZED VIEW IF EXISTS items_by_price_and_producer 
"""
session.execute(query)

query = """
    CREATE MATERIALIZED VIEW items_by_price_and_producer 
    AS SELECT * 
    FROM items 
    WHERE price is not null and category is not null and producer is not null
    PRIMARY KEY ((price, producer), category)
"""
result = session.execute(query)

In [321]:
query = """
    select * 
    from items_by_price_and_producer
    where price = 1000 and producer = 'Apple'
"""
result = session.execute(query)

In [322]:
for row in result.all():
    print(row)

Row(price=1000, producer='Apple', category='Phone', custom_params=None, id=UUID('f1b2c097-dddb-11ee-845d-1062e54e838c'), model='iPhone 12')


In [323]:
query = """
    select * 
    from items
    where custom_params CONTAINS KEY 'max_discount'
"""
result = session.execute(query)

In [324]:
for row in result.all():
    print(row)

Row(category='TV', price=2000, custom_params=OrderedMapSerializedKey([('max_discount', '25%')]), id=UUID('f1b6d06a-dddb-11ee-ab99-1062e54e838c'), model='Gigantic 1000', producer='Spacio')


In [325]:
query = """
    select * 
    from items
    where custom_params['max_discount'] = '25%'
"""
result = session.execute(query)

In [326]:
for row in result.all():
    print(row)

Row(category='TV', price=2000, custom_params=OrderedMapSerializedKey([('max_discount', '25%')]), id=UUID('f1b6d06a-dddb-11ee-ab99-1062e54e838c'), model='Gigantic 1000', producer='Spacio')


In [329]:
query = """
    UPDATE items
    set custom_params['max_discount'] = '30%'
    where category = 'TV' and price = 2000
"""
result = session.execute(query)

In [335]:
query = """
    select * 
    from items
    where category = 'TV' and price = 2000
"""
result = session.execute(query)

In [336]:
for row in result.all():
    print(row)

Row(category='TV', price=2000, custom_params=OrderedMapSerializedKey([('max_discount', '30%'), ('min_discount', '10%')]), id=UUID('f1b6d06a-dddb-11ee-ab99-1062e54e838c'), model='Gigantic 1000', producer='Spacio')


In [341]:
query = """
    UPDATE items
    set custom_params['min_discount'] = '10%'
    where category = 'TV' and price = 2000
"""
result = session.execute(query)

In [343]:
query = """
    select * 
    from items
    where category = 'TV' and price = 2000
"""
result = session.execute(query)

In [344]:
for row in result.all():
    print(row)

Row(category='TV', price=2000, custom_params=OrderedMapSerializedKey([('max_discount', '30%'), ('min_discount', '10%')]), id=UUID('f1b6d06a-dddb-11ee-ab99-1062e54e838c'), model='Gigantic 1000', producer='Spacio')


In [345]:
query = """
    DELETE custom_params['min_discount']
    from items
    where category = 'TV' and price = 2000
"""
result = session.execute(query)

In [346]:
query = """
    select * 
    from items
    where category = 'TV' and price = 2000
"""
result = session.execute(query)

In [347]:
for row in result.all():
    print(row)

Row(category='TV', price=2000, custom_params=OrderedMapSerializedKey([('max_discount', '30%')]), id=UUID('f1b6d06a-dddb-11ee-ab99-1062e54e838c'), model='Gigantic 1000', producer='Spacio')


In [351]:
query = """
    select * 
    from items
"""
result = session.execute(query)
for row in result.all():
    print(row)

Row(category='Phone', price=1000, custom_params=None, id=UUID('f1b2c097-dddb-11ee-845d-1062e54e838c'), model='iPhone 12', producer='Apple')
Row(category='Phone', price=1150, custom_params=None, id=UUID('f1b3861b-dddb-11ee-8604-1062e54e838c'), model='Galaxy S26', producer='Samsung')
Row(category='Phone', price=1200, custom_params=None, id=UUID('f1b17adc-dddb-11ee-b6a4-1062e54e838c'), model='iPhone 13', producer='Apple')
Row(category='TV', price=1200, custom_params=OrderedMapSerializedKey([('sub_category', 'Smart TV')]), id=UUID('f1b449f2-dddb-11ee-be16-1062e54e838c'), model='SmartTV v4.2', producer='Samsung')
Row(category='TV', price=2000, custom_params=OrderedMapSerializedKey([('max_discount', '30%')]), id=UUID('f1b6d06a-dddb-11ee-ab99-1062e54e838c'), model='Gigantic 1000', producer='Spacio')
Row(category='Cooker', price=2000, custom_params=OrderedMapSerializedKey([('height', '38cm')]), id=UUID('f1b79404-dddb-11ee-b0a5-1062e54e838c'), model='Pasteroni', producer='Italiez')
Row(category

In [361]:
orders = [
    {
        "date": datetime.today().date(),
        "total_sum": 2300,
        "customer_name": "Mike Wazowsky",
        "items_id" : [uuid.UUID("f1b2c097-dddb-11ee-845d-1062e54e838c"), uuid.UUID("f1b3861b-dddb-11ee-8604-1062e54e838c")]
    },
    {
        "date": datetime.today().date() - timedelta(days=1),
        "total_sum": 5900,
        "customer_name" : "Mike Wazowsky",
        "items_id" : [uuid.UUID("f1b2c097-dddb-11ee-845d-1062e54e838c"), uuid.UUID("f1b17adc-dddb-11ee-b6a4-1062e54e838c")]
    },
    {
        "date": datetime.today().date() - timedelta(days=3),
        "total_sum": 5900,
        "customer_name": "James Sullivan",
        "items_id" : [uuid.UUID("f1b449f2-dddb-11ee-be16-1062e54e838c")]
    },
    {
        "date": datetime.today().date() - timedelta(days=5),
        "total_sum": 4200,
        "customer_name" :  "Randall Boggs",
        "items_id" : [uuid.UUID("f1b6d06a-dddb-11ee-ab99-1062e54e838c")]
    },
]

In [489]:

query = """
    DROP TABLE IF EXISTS orders
"""
result = session.execute(query)

In [490]:

query = """
    CREATE TABLE IF NOT EXISTS orders (
        id UUID,
        date date,
        total_sum int,
        customer_name text,
        items_id list<UUID>,
        PRIMARY KEY (customer_name, date)
    );
"""
session.execute(query)

In [491]:
query = """
    INSERT INTO orders (id, date, total_sum, customer_name, items_id)
    VALUES (%s, %s, %s, %s, %s)
"""
for order in orders:
    session.execute(query, (uuid.uuid1(), *order.values()))

In [492]:
query = """
    DESCRIBE table orders
"""
result = session.execute(query)

In [493]:
for row in result.all():
    print(row)

Row(keyspace_name='test_keyspace', type='table', name='orders', create_statement="CREATE TABLE test_keyspace.orders (\n    customer_name text,\n    date date,\n    id uuid,\n    total_sum int,\n    items_id list<uuid>,\n    PRIMARY KEY (customer_name, date)\n) WITH CLUSTERING ORDER BY (date ASC)\n    AND additional_write_policy = '99p'\n    AND bloom_filter_fp_chance = 0.01\n    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}\n    AND cdc = false\n    AND comment = ''\n    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}\n    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}\n    AND memtable = 'default'\n    AND crc_check_chance = 1.0\n    AND default_time_to_live = 0\n    AND extensions = {}\n    AND gc_grace_seconds = 864000\n    AND max_index_interval = 2048\n    AND memtable_flush_period_in_ms = 0\n    AND min_index_interval 

In [494]:
query = """
    SELECT *
    FROM orders
    where customer_name = 'Mike Wazowsky'
    order by date
"""
result = session.execute(query)

In [495]:
for row in result.all():
    print(row)

Row(customer_name='Mike Wazowsky', date=Date(19790), id=UUID('b365928a-ddef-11ee-8938-1062e54e838c'), items_id=[UUID('f1b2c097-dddb-11ee-845d-1062e54e838c'), UUID('f1b17adc-dddb-11ee-b6a4-1062e54e838c')], total_sum=5900)
Row(customer_name='Mike Wazowsky', date=Date(19791), id=UUID('b363e361-ddef-11ee-906d-1062e54e838c'), items_id=[UUID('f1b2c097-dddb-11ee-845d-1062e54e838c'), UUID('f1b3861b-dddb-11ee-8604-1062e54e838c')], total_sum=2300)


In [496]:
query = """
    CREATE INDEX IF NOT EXISTS ON orders (items_id)
"""
session.execute(query)

In [497]:
query = """
    SELECT *
    FROM orders
    where customer_name = 'Mike Wazowsky' and items_id CONTAINS f1b17adc-dddb-11ee-b6a4-1062e54e838c
"""
result = session.execute(query)

In [498]:
for row in result.all():
    print(row)

Row(customer_name='Mike Wazowsky', date=Date(19790), id=UUID('b365928a-ddef-11ee-8938-1062e54e838c'), items_id=[UUID('f1b2c097-dddb-11ee-845d-1062e54e838c'), UUID('f1b17adc-dddb-11ee-b6a4-1062e54e838c')], total_sum=5900)


In [499]:
query = """
    SELECT *
    FROM orders
    where customer_name = 'Mike Wazowsky' and date >= '2024-03-01' and date <= '2024-03-08'
    order by date
"""
result = session.execute(query)

In [500]:
for row in result.all():
    print(row)

Row(customer_name='Mike Wazowsky', date=Date(19790), id=UUID('b365928a-ddef-11ee-8938-1062e54e838c'), items_id=[UUID('f1b2c097-dddb-11ee-845d-1062e54e838c'), UUID('f1b17adc-dddb-11ee-b6a4-1062e54e838c')], total_sum=5900)


In [501]:
query = """
    SELECT count(*)
    FROM orders
    where customer_name = 'Mike Wazowsky' and date >= '2024-03-01' and date <= '2024-03-08'
    order by date
"""
result = session.execute(query)

In [502]:
for row in result.all():
    print(row)

Row(count=1)


In [503]:
query = """
    SELECT customer_name, sum(total_sum)
    FROM orders
    GROUP BY customer_name
"""
result = session.execute(query)

In [504]:
for row in result.all():
    print(row)

Row(customer_name='Mike Wazowsky', system_sum_total_sum=8200)
Row(customer_name='James Sullivan', system_sum_total_sum=5900)
Row(customer_name='Randall Boggs', system_sum_total_sum=4200)


In [505]:
query = """
    SELECT customer_name, id, max(total_sum)
    FROM orders
    GROUP BY customer_name
"""
result = session.execute(query)

In [506]:
for row in result.all():
    print(row)

Row(customer_name='Mike Wazowsky', id=UUID('b365928a-ddef-11ee-8938-1062e54e838c'), system_max_total_sum=5900)
Row(customer_name='James Sullivan', id=UUID('b3663894-ddef-11ee-8d89-1062e54e838c'), system_max_total_sum=5900)
Row(customer_name='Randall Boggs', id=UUID('b3670fbe-ddef-11ee-a27a-1062e54e838c'), system_max_total_sum=4200)


In [507]:
query = """
    UPDATE orders
    set items_id = [f1b449f2-dddb-11ee-be16-1062e54e838c] + items_id,
    total_sum = 4200 + 1200
    where customer_name = 'Randall Boggs' and date = '2024-03-04'
"""
result = session.execute(query)

In [508]:
query = """
    SELECT *
    FROM orders
    where customer_name = 'Randall Boggs'
"""
result = session.execute(query)

In [509]:
for row in result.all():
    print(row)

Row(customer_name='Randall Boggs', date=Date(19786), id=UUID('b3670fbe-ddef-11ee-a27a-1062e54e838c'), items_id=[UUID('f1b449f2-dddb-11ee-be16-1062e54e838c'), UUID('f1b6d06a-dddb-11ee-ab99-1062e54e838c')], total_sum=5400)


In [512]:
query = """
    SELECT id, WRITETIME(total_sum)
    FROM orders
"""
result = session.execute(query)

In [513]:
for row in result.all():
    print(row)

Row(id=UUID('b365928a-ddef-11ee-8938-1062e54e838c'), writetime_total_sum=1709973207176462)
Row(id=UUID('b363e361-ddef-11ee-906d-1062e54e838c'), writetime_total_sum=1709973207165424)
Row(id=UUID('b3663894-ddef-11ee-8d89-1062e54e838c'), writetime_total_sum=1709973207180712)
Row(id=UUID('b3670fbe-ddef-11ee-a27a-1062e54e838c'), writetime_total_sum=1709973217868573)


In [514]:
order = {
    "date": datetime.today().date() - timedelta(days=2),
    "total_sum": 2500,
    "customer_name" :  "Randall Boggs",
    "items_id" : [uuid.UUID("f1b858f3-dddb-11ee-8b0d-1062e54e838c")]
}

In [515]:
query = """
    INSERT INTO orders (id, date, total_sum, customer_name, items_id)
    VALUES (%s, %s, %s, %s, %s)
    USING TTL 86400
"""
session.execute(query, (uuid.uuid1(), *order.values()))

In [516]:
query = """
    SELECT *
    FROM orders
"""
result = session.execute(query)

In [517]:
for row in result.all():
    print(row)

Row(customer_name='Mike Wazowsky', date=Date(19790), id=UUID('b365928a-ddef-11ee-8938-1062e54e838c'), items_id=[UUID('f1b2c097-dddb-11ee-845d-1062e54e838c'), UUID('f1b17adc-dddb-11ee-b6a4-1062e54e838c')], total_sum=5900)
Row(customer_name='Mike Wazowsky', date=Date(19791), id=UUID('b363e361-ddef-11ee-906d-1062e54e838c'), items_id=[UUID('f1b2c097-dddb-11ee-845d-1062e54e838c'), UUID('f1b3861b-dddb-11ee-8604-1062e54e838c')], total_sum=2300)
Row(customer_name='James Sullivan', date=Date(19788), id=UUID('b3663894-ddef-11ee-8d89-1062e54e838c'), items_id=[UUID('f1b449f2-dddb-11ee-be16-1062e54e838c')], total_sum=5900)
Row(customer_name='Randall Boggs', date=Date(19786), id=UUID('b3670fbe-ddef-11ee-a27a-1062e54e838c'), items_id=[UUID('f1b449f2-dddb-11ee-be16-1062e54e838c'), UUID('f1b6d06a-dddb-11ee-ab99-1062e54e838c')], total_sum=5400)
Row(customer_name='Randall Boggs', date=Date(19789), id=UUID('2ad9a39d-ddf0-11ee-bddb-1062e54e838c'), items_id=[UUID('f1b858f3-dddb-11ee-8b0d-1062e54e838c')], to

In [518]:
query = """
    SELECT json *
    FROM orders
"""
result = session.execute(query)

In [519]:
for row in result.all():
    print(row)

Row(json='{"customer_name": "Mike Wazowsky", "date": "2024-03-08", "id": "b365928a-ddef-11ee-8938-1062e54e838c", "items_id": ["f1b2c097-dddb-11ee-845d-1062e54e838c", "f1b17adc-dddb-11ee-b6a4-1062e54e838c"], "total_sum": 5900}')
Row(json='{"customer_name": "Mike Wazowsky", "date": "2024-03-09", "id": "b363e361-ddef-11ee-906d-1062e54e838c", "items_id": ["f1b2c097-dddb-11ee-845d-1062e54e838c", "f1b3861b-dddb-11ee-8604-1062e54e838c"], "total_sum": 2300}')
Row(json='{"customer_name": "James Sullivan", "date": "2024-03-06", "id": "b3663894-ddef-11ee-8d89-1062e54e838c", "items_id": ["f1b449f2-dddb-11ee-be16-1062e54e838c"], "total_sum": 5900}')
Row(json='{"customer_name": "Randall Boggs", "date": "2024-03-04", "id": "b3670fbe-ddef-11ee-a27a-1062e54e838c", "items_id": ["f1b449f2-dddb-11ee-be16-1062e54e838c", "f1b6d06a-dddb-11ee-ab99-1062e54e838c"], "total_sum": 5400}')
Row(json='{"customer_name": "Randall Boggs", "date": "2024-03-07", "id": "2ad9a39d-ddf0-11ee-bddb-1062e54e838c", "items_id": ["

In [520]:
import json

In [536]:
order = {
    "id": str(uuid.uuid1()),
    "date": str(datetime.today().date() - timedelta(days=1)),
    "total_sum": 2300,
    "customer_name" :  "Randall Roggs",
    "items_id" : [str(uuid.UUID("f1b858f3-dddb-11ee-8b0d-1062e54e838c"))]
}

In [540]:
query = f"""
    INSERT INTO orders JSON '{json.dumps(order)}'
"""
session.execute(query)

In [541]:
query = """
    SELECT json *
    FROM orders
"""
result = session.execute(query)

In [542]:
for row in result.all():
    print(row)

Row(json='{"customer_name": "Mike Wazowsky", "date": "2024-03-08", "id": "b365928a-ddef-11ee-8938-1062e54e838c", "items_id": ["f1b2c097-dddb-11ee-845d-1062e54e838c", "f1b17adc-dddb-11ee-b6a4-1062e54e838c"], "total_sum": 5900}')
Row(json='{"customer_name": "Mike Wazowsky", "date": "2024-03-09", "id": "b363e361-ddef-11ee-906d-1062e54e838c", "items_id": ["f1b2c097-dddb-11ee-845d-1062e54e838c", "f1b3861b-dddb-11ee-8604-1062e54e838c"], "total_sum": 2300}')
Row(json='{"customer_name": "James Sullivan", "date": "2024-03-06", "id": "b3663894-ddef-11ee-8d89-1062e54e838c", "items_id": ["f1b449f2-dddb-11ee-be16-1062e54e838c"], "total_sum": 5900}')
Row(json='{"customer_name": "Randall Roggs", "date": "2024-03-08", "id": "fdf658a9-ddf0-11ee-88c5-1062e54e838c", "items_id": ["f1b858f3-dddb-11ee-8b0d-1062e54e838c"], "total_sum": 2300}')
Row(json='{"customer_name": "Randall Boggs", "date": "2024-03-04", "id": "b3670fbe-ddef-11ee-a27a-1062e54e838c", "items_id": ["f1b449f2-dddb-11ee-be16-1062e54e838c", "